# Motion Control Module

Once the vision module is implemented to send the robot's position and angle, and the optimal path is computed, the motion control module comes into play to guide the robot toward its goal efficiently and reliably. Our A* algorithm computes a highly precise path, which we segment into closely spaced waypoints to ensure smooth and accurate motion. To navigate these waypoints, we implemented a simplified version of the Astolfi controller, which is ideal for the Thymio's non-holonomic differential drive system. This controller focuses on minimizing two critical variables: the distance to the goal ($\rho$) and the alignment error ($\alpha$), ensuring the robot aligns itself with its path and reaches each waypoint effectively.

![Controller Diagram](/controller.jpg)

The distance to the goal, $\rho$, is calculated as:

$$
\rho = \sqrt{(x_{goal} - x_{robot})^2 + (y_{goal} - y_{robot})^2}
$$

This represents the Euclidean distance between the robot's position and the target waypoint. 

The alignment error, $\alpha$, is defined as:

$$
\alpha = \arctan2(dy, dx) - \theta
$$

Here, $dx = x_{goal} - x_{robot}$ and $dy = y_{goal} - y_{robot}$, while $\theta$ is the robot's current orientation. The term $\arctan2(dy, dx)$ gives the direction to the waypoint relative to the robot's current position, and subtracting $\theta$ calculates the angular difference. To adapt to the inverted y-axis of the camera’s coordinate frame, we flipped the signs of the y-coordinates in our implementation, ensuring consistency in how the controller interprets these values.

The controller generates two outputs: linear velocity ($v$) and angular velocity ($\omega$), which are computed using proportional control laws. These are defined as:

$$
v = K_\rho \cdot \rho
$$

$$
\omega = K_\alpha \cdot \alpha
$$

Here, $K_\rho > 0$ and $K_\alpha > 0$ are gain parameters tuned for stability. The gain $K_\alpha$ is chosen to be greater than $K_\rho$ ($K_\alpha > K_\rho$) to ensure that the robot prioritizes alignment before moving forward, preventing overshooting or instability. Both $K_\rho$ and $K_\alpha$ must be positive to ensure the robot moves toward the goal while aligning itself correctly, maintaining stability by progressively reducing position and angular errors, as guaranteed by Lyapunov’s method.

To determine these gains, we used a simple method for initial parameter estimation. Starting with a straightforward scenario where the robot moves in a straight line, such as from (0,0,0) to (x,0), we defined a desired wheel speed for this motion and estimated the corresponding $v$ and $\rho$. From this, we calculated a starting value for $K_\rho$ to produce the required linear velocity. For $K_\alpha$, we chose a value slightly larger than $K_\rho$ to ensure the robot prioritizes angular alignment without causing instability. These initial guesses were then refined through empirical tuning, where we tested different scenarios, like turning or navigating along curved paths, and adjusted the parameters until the robot exhibited stable and smooth behavior.This iterative approach proved practical and effective for our implementation.

These velocities are then mapped to the Thymio's motor speeds in `compute_motor_speeds` function using the differential drive equations:

$$
v_{right} = v + \omega \cdot d
$$

$$
v_{left} = v - \omega \cdot d
$$

`compute_motor_speeds` also makes sure the velocities don't exceed a certain max_speed we precised, and converts them to thymio speed units.

Here, $d$ represents the distance between the robot's wheels. This mapping ensures that the Thymio can translate the computed velocities into physical movement. Due to the short distances between waypoints and the simplicity of the paths, we implemented a streamlined version of the Astolfi controller, omitting the beta angle correction. A certain threshold is defined for the distance to the goal ($\rho$), which determines when the Thymio has successfully reached its target waypoint. By continuously recalculating the distance and alignment errors as the robot moves, the controller dynamically adjusts the velocities, enabling the Thymio to navigate its path with accuracy and stability. This method combines the precision of proportional control with the adaptability required for real-world navigation.


### Local Navigation

We initially tried to implement a wall-following technique for obstacle avoidance, but due to the nature of our map—specifically the narrow paths the Thymio has to navigate—we opted for a different approach that suited our environment better. To handle these situations more effectively, we decided to make our Thymio “smarter” by leveraging the **global map** we defined earlier. 

When computing the global map, we generate a **distance map**. This distance map converts our environment into a grid of distances, where each cell represents how far it is from the nearest wall or object in the map. Essentially, it gives the Thymio a better sense of the available space around it in a precomputed, global way, rather than relying solely on its proximity sensors. This makes navigation in tight spaces much easier and more efficient.

When the Thymio detects a local 3D obstacle (using its proximity sensors), it enters the **avoidance state**. In this state, the decision on how to avoid the obstacle is based on determining which side (left or right) has more free space. This decision is made by the `check_sides` function, which uses the distance map to evaluate the distance to obstacles on either side of the robot. Specifically, it projects left and right from the robot’s current position, checking where there is more space to turn. This is a simple yet effective way to choose the best side to maneuver toward.

In **avoidance mode**, the Thymio performs a straightforward maneuver in `avoid_obstacle` function: it turns toward the side with more space, reorients itself, and then tries to get back on track. Once the obstacle is cleared, the Thymio recomputes the path to its goal using our A* algorithm. This ensures it resumes its journey efficiently, without getting stuck or lost.

This approach allowed us to better handle narrow paths and complex obstacles while keeping the logic simple and adaptive to the challenges of our map.